# Foursquare API with Python Exploratory Notebook

This is a notebook that explores some of the features of the Foursquare API.

We're using Python for the coding.  We'll be using the foursquare API (and Python) in the Battle of the Neighborhoods notebook as part of the IBM Data Science capstone project.

Some of the features this notebook shows:

* Foursquare API search function
* Exploring venues
* Exploring users
* Foursquare API explore function
* Trending venues

## Importing Necessary Libraries and Defining Foursquare Credentials

In [1]:
import requests
import pandas as pd
import numpy as np
import random

# Module to convert addresses into latitude and longitude
!pip install geopy
from geopy.geocoders import Nominatim 

# Image display libraries
from IPython.display import Image
from IPython.core.display import HTML

# Transforming json into pandas dataframes
from pandas.io.json import json_normalize

# Folium mapping library
import folium

In [2]:
# Connecting to foursquare

CLIENT_ID = 'P4GV2G4LEKK4XMTHM0H4H5W3CFR055TAHR2IQ3JPLJ0TFHEK'
CLIENT_SECRET = 'CJ2YTC5JCO4F3PNHKPSZPMFOTIU5RYD4QD1WB5FYRCS4FD5B'
VERSION ='20180604'
LIMIT = 30

## Search for a Specific Venue

Assume we are staying at the Conrad Hotel.  We begin by converting the address to latitude and longitude.

To do this we need to create an instance of the geocoder, and define a user_agent.

In [3]:
address = '102 North End Ave, New York, NY'
geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [4]:
search_query = 'Italian'
radius = 500

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=P4GV2G4LEKK4XMTHM0H4H5W3CFR055TAHR2IQ3JPLJ0TFHEK&client_secret=CJ2YTC5JCO4F3PNHKPSZPMFOTIU5RYD4QD1WB5FYRCS4FD5B&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb04bef4450ba020e8f9d21'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-16053892

In [7]:
# Assign relevant parts of the response to venues
venues = results['response']['venues']

# Put this into a pandas dataframe
df = pd.json_normalize(venues)
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1605389295,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1605389295,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1605389295,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


In [8]:
# Perform some cleaning on this data

filtered_columns = ['name','categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]

# Extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis = 1)

df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

df_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [9]:
df_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [10]:
venues_map = folium.Map(location = [latitude, longitude], zoom_start = 13)

folium.CircleMarker(
    [latitude, longitude],
    radius = 10,
    color = 'red',
    popup = 'Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6).add_to(venues_map)

for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'blue',
        popup = label,
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6).add_to(venues_map)

venues_map

## Exploring a Particular Venue

Now we use the foursquare API to explore a particular venue.  In this case, we'll explore the closest italian restaurant to our hotel: Harry's Italian Pizza Bar

In [11]:
venue_id = '4fa862b3e4b0ebff2f749f06'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=P4GV2G4LEKK4XMTHM0H4H5W3CFR055TAHR2IQ3JPLJ0TFHEK&client_secret=CJ2YTC5JCO4F3PNHKPSZPMFOTIU5RYD4QD1WB5FYRCS4FD5B&v=20180604'

In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'n

In [13]:
# Get some information on Harry's

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [14]:
# Check the rating of the second closest italian restaurant

venue_id = '4f3232e219836c91c7bfde94'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [15]:
# Check the rating of the third closest italian restaurant

venue_id = '3fd66200f964a520f4e41ee3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


Since the third closest italian restaurant has the highest rating amongst the three closest restaurants, let's use the foursquare API to explore further.

In [17]:
result['response']['venue']['tips']['count']

19

In [29]:
# Retrieve the tips from the API

limit = 15
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb051534450ba020ea4b0dc'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 5,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'isSanctioned': False, 'firstName': 'Nick', 'lastName': 'E'},
     'authorInteractionType': 'liked'}]}}}

In [30]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [31]:
pd.set_option('display.max_colwidth',None)
tips_df = pd.json_normalize(tips)

filtered_columns = ['text','agreeCount','disagreeCount','id','user.firstName','user.lastName','user.id']
tips_filtered = tips_df.reindex(columns = filtered_columns) # Replacement reindexing method, April 2020
#tips_filtered = tips_df.loc[:,filtered_columns]  *** This method of reindexing is deprecated as of April 2020

tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",5,0,5ab1cb46c9a517174651d3fe,Nick,E,NaN


## Search a Particular Foursquare User

Next we use the API to find information about a particular user on foursquare.

In [33]:
idnumber = '48454224'

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION)

results = requests.get(url).json()
user_data = results['response']['tips']['items'][0]['venue']['photos']['groups'][0]['items']
pd.set_option('display.max_columns',None)

users_df = pd.json_normalize(user_data)

filtered_columns = ['id','user.id','user.firstName','user.lastName']
tips_filtered = users_df.reindex(columns = filtered_columns)

df = tips_filtered
df

,id,user.id,user.firstName,user.lastName
0,57099084498e6e5b94d5a312,NaN,Cesar,H
1,541e1d3b498e8e793290f7cd,NaN,Cesar,H
2,53cb1c82498ef31f2f2eba34,NaN,Marcs,M


In [39]:
g = df.loc[df['id'] == '57099084498e6e5b94d5a312']
print('First name: ' + g['user.firstName'])
print('Last name: ' + g['user.lastName'])

0    First name: Cesar
Name: user.firstName, dtype: object
0    Last name: H
Name: user.lastName, dtype: object


In [40]:
# Retrieve user's profile image

Image(url='https://fastly.4sqi.net/img/general/540x920/133773133_ODR5Au05ENkSyu3xxfV3VOfCa0idDfm9Q4n8YbhZDRQ.jpg')

In [41]:
user_id = '484542633'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']

pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips)

filtered_columns = ['text','agreeCount','disagreeCount','id']
tips_filtered = tips_df.reindex(columns = filtered_columns)

tips_filtered

,text,agreeCount,disagreeCount,id
0,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
1,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",2,0,5acbec70a0215b732e264fe8
2,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
3,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
4,Burger game strong 💪,1,0,5ab575fb6bdee65f759da8c1
5,"Great burgers & fries! Also, this place is exactly what it’s like when you go to a bar in the Southwest. Source: I’m from Arizona.",2,0,5ab5575d73fe2516ad8f363b
6,Açaí bowl + peanut butter + whey protein = 💪💪💪,1,0,5ab42db53c858d64af2688a4
7,Highly underrated and way less crowded than Central Park!,3,0,5ab42c396f706a29f53ad1a8
8,Get the açaí bowl with peanut butter after your work out and thank me later 👌,1,0,5ab42aca2a7ab6333652b266
9,"When you want a burger, this should be the first thing that comes to mind. A+!",1,0,5ab42a28da5e5617d18e3a6a


In [44]:
# Get venue for the tip with greatest number of agrees

tip_id = '5ab5575d73fe2516ad8f363b'

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
print(result['response']['tips']['items'][0]['venue']['name'])
print(result['response']['tips']['items'][0]['venue']['location'])

Tres Bar
{'lat': 20.17693758560229, 'lng': -98.04832423694748, 'labeledLatLngs': [{'label': 'display', 'lat': 20.17693758560229, 'lng': -98.04832423694748}], 'cc': 'MX', 'country': 'México', 'formattedAddress': ['México']}


## Exploring a Particular Location with Foursquare API

In [45]:
latitude = 40.715337
longitude = -74.008848

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=P4GV2G4LEKK4XMTHM0H4H5W3CFR055TAHR2IQ3JPLJ0TFHEK&client_secret=CJ2YTC5JCO4F3PNHKPSZPMFOTIU5RYD4QD1WB5FYRCS4FD5B&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

In [46]:
results = requests.get(url).json()
'There are {} around ECCO restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around ECCO restaurant.'

In [47]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d5f24ec09484500079aee00',
  'name': 'Los Tacos No. 1',
  'location': {'address': '136 Church St',
   'lat': 40.714267,
   'lng': -74.008756,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.714267,
     'lng': -74.008756}],
   'distance': 119,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['136 Church St',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d151941735',
    'name': 'Taco Place',
    'pluralName': 'Taco Places',
    'shortName': 'Tacos',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '2180700',
   'url': 'https://www.seamless.com/menu/los-tacos-no-1-tribeca-136-church-st-new-york/21

In [48]:
df = pd.json_normalize(items)

filtered_columns = ['venue.name', 'venue.categories'] + [col for col in df.columns if col.startswith('venue.location.')] + ['venue.id']
df_filtered = df.loc[:,filtered_columns]

df_filtered['venue.categories'] = df_filtered.apply(get_category_type, axis = 1)

df_filtered.columns = [col.split('.')[-1] for col in df_filtered.columns]

df_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",NaN,NaN,5d5f24ec09484500079aee00
1,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",Church St,Tribeca,4af5d65ff964a52091fd21e3
2,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",at Church St,NaN,4c154c9a77cea593c401d260
3,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",btwn Greenwich St & W Broadway,NaN,54148bc6498ea7bb8c05b70a
4,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}, {'label': 'entrance', 'lat': 40.716508, 'lng': -74.007989}]",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",btwn W Broadway & Church St,NaN,4a8f2f39f964a520471420e3
5,Chambers Street Wines,Wine Shop,148 Chambers St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}, {'label': 'entrance', 'lat': 40.715696, 'lng': -74.00988}]",88,10007,US,New York,NY,United States,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]",btwn West Broadway & Hudson St,NaN,4adcf23cf964a520cc6221e3
6,Heyday,Spa,92 Reade St,40.715726,-74.007767,"[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}, {'label': 'entrance', 'lat': 40.715654, 'lng': -74.00782}]",100,10013,US,New York,NY,United States,"[92 Reade St, New York, NY 10013, United States]",NaN,NaN,57ad129c498e05b086594d72
7,Lekka Burger,Burger Joint,81 Warren St,40.715246,-74.010559,"[{'label': 'display', 'lat': 40.715246, 'lng': -74.010559}]",144,10007,US,New York,NY,United States,"[81 Warren St (btw Greenwich & West Broadway), New York, NY 10007, United States]",btw Greenwich & West Broadway,NaN,5dc6f6a5ea8dfb00080f6faa
8,Equinox Tribeca,Gym,54 Murray St,40.714099,-74.009686,"[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",154,10007,US,New York,NY,United States,"[54 Murray St (at W Broadway), New York, NY 10007, United States]",at W Broadway,NaN,4a6e331af964a52031d41fe3
9,Nish Nūsh,Falafel Restaurant,88 Reade St,40.715537,-74.007725,"[{'label': 'display', 'lat': 40.71553710116416, 'lng': -74.00772452925565}, {'label': 'entrance', 'lat': 40.715615, 'lng': -74.00773}]",97,10013,US,New York,NY,United States,"[88 Reade St (at Church St), New York, NY 10013, United States]",at Church St,NaN,50ba9119e4b071a4bae6dc10


In [49]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start = 15)

folium.CircleMarker(
    [latitude, longitude],
    radius = 10,
    popup = 'ECCO',
    fill = True,
    color = 'red',
    fill_color = 'red',
    fill_opacity = 0.6,
    ).add_to(venues_map)

for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        fill = True,
        color = 'blue',
        fill_color = 'blue',
        fill_opacity = 0.6).add_to(venues_map)

venues_map

## Exploring Trending Venues

In [50]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb060c3d64a734725e2e20e'},
 'response': {'venues': [{'id': '5d31423e4918880009c2b26d',
    'name': 'Ray’s Bar',
    'location': {'address': '177 Chrystie St',
     'lat': 40.721123,
     'lng': -73.992581,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.721123,
       'lng': -73.992581}],
     'distance': 1516,
     'postalCode': '10002',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['177 Chrystie St',
      'New York, NY 10002',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d116941735',
      'name': 'Bar',
      'pluralName': 'Bars',
      'shortName': 'Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'primary': True}]}]}}

In [51]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'

else:
    trending_venues = results['response']['venues']
    trending_venues_df = pd.json_normalize(trending_venues)
    
    columns_filtered = ['name','categories'] + ['location.distance','location.city','location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.reindex(columns = columns_filtered)

    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis = 1)

In [52]:
trending_venues_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Ray’s Bar,Bar,1516,New York,10002,NY,United States,40.721123,-73.992581


In [53]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'
    
else:
    venues_map = folium.Map(location=[latitude,longitude], zoom_start = 15)
    
    folium.CircleMarker(
        [latitude, longitude],
        radius = 10,
        popup = 'Ecco',
        fill = True,
        color = 'red',
        fill_color = 'red',
        fill_opacity = 0.6).add_to(venues_map)
    
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat,lng],
            radius = 5,
            popup = label,
            fill = True,
            color = 'blue',
            fill_color = 'blue',
            fill_opacity = 0.6).add_to(venues_map)

venues_map